In [1]:
from vgg16 import vgg16 as vgg_16
from skimage.transform import resize
from skimage.io import imread, imshow
import numpy as np
import tensorflow as tf
from google.protobuf import json_format
from imagenet_classes import class_names
import re

# It wrong follow thw paper you need to implement the model ad use back prop to image DAAAAAAA

In [2]:
tf.reset_default_graph()
with tf.Session() as sess:
    imgs = tf.placeholder(tf.float32,[1, 224, 224, 3])
    vgg = vgg_16(imgs, 'vgg16_weights.npz', sess)
    vgg_graph = sess.graph
    

Loading weights ['conv1_1_W', 'conv1_1_b', 'conv1_2_W', 'conv1_2_b', 'conv2_1_W', 'conv2_1_b', 'conv2_2_W', 'conv2_2_b', 'conv3_1_W', 'conv3_1_b', 'conv3_2_W', 'conv3_2_b', 'conv3_3_W', 'conv3_3_b', 'conv4_1_W', 'conv4_1_b', 'conv4_2_W', 'conv4_2_b', 'conv4_3_W', 'conv4_3_b', 'conv5_1_W', 'conv5_1_b', 'conv5_2_W', 'conv5_2_b', 'conv5_3_W', 'conv5_3_b', 'fc6_W', 'fc6_b', 'fc7_W', 'fc7_b', 'fc8_W', 'fc8_b']


In [3]:
ac_convs=list()
for v in vgg_graph.get_collection("trainable_variables"):
    if "biases" not in v.name:
        l_name = re.findall(r".+(?=\/)", v.name)[0]
        if l_name.startswith("conv"):
            ac_convs.append(vgg_graph.get_tensor_by_name(l_name + ":0"))
        print("name:{:8} kernel:{:15}".format(l_name, str(v.shape)))


name:conv1_1  kernel:(3, 3, 3, 64)  
name:conv1_2  kernel:(3, 3, 64, 64) 
name:conv2_1  kernel:(3, 3, 64, 128)
name:conv2_2  kernel:(3, 3, 128, 128)
name:conv3_1  kernel:(3, 3, 128, 256)
name:conv3_2  kernel:(3, 3, 256, 256)
name:conv3_3  kernel:(3, 3, 256, 256)
name:conv4_1  kernel:(3, 3, 256, 512)
name:conv4_2  kernel:(3, 3, 512, 512)
name:conv4_3  kernel:(3, 3, 512, 512)
name:conv5_1  kernel:(3, 3, 512, 512)
name:conv5_2  kernel:(3, 3, 512, 512)
name:conv5_3  kernel:(3, 3, 512, 512)
name:fc1      kernel:(25088, 4096)  
name:fc2      kernel:(4096, 4096)   
name:fc3      kernel:(4096, 1000)   


In [ ]:
content = resize(imread("sh.jpg"), [224, 224])
style = resize(imread("salvador-dali-painting.jpg"), [224, 224])

In [6]:
vgg.conv3_3

<tf.Tensor 'conv3_3:0' shape=(1, 56, 56, 256) dtype=float32>

In [98]:
def get_content_loss(content_act, generated_act):
    return(0.5*np.sum(np.square(content_act - generated_act)))

def get_content_loss_gradient(content_act, generated_act):
    grad = content_act - generated_act
    grad[np.where(content_act<0)] = 0
    return(grad)

def get_gram(content_act):
    assert(len(content_act.shape) == 3)
    assert(content_act.shape[0] == content_act.shape[1])
    gram = content_act.reshape((content_act.shape[0]*content_act.shape[1], content_act.shape[2]))
    return(gram.T.dot(gram))

def get_style_El(content_gram, generated_gram, no_of_filters, filter_size):
    El = np.sum(np.square(generated_gram - content_gram))/(4*(no_of_filters**2)*(filter_size**2))
    return(El)

def get_style_El_gradient()
    
    
    

# Testing the content

In [110]:

np.random.seed(0)
content_act = np.random.uniform(size=(10, 10)).astype(np.float32) - 0.5
np.random.seed(1)
generated_act = np.random.uniform(size=(10, 10)).astype(np.float32) - 0.5

assert(np.sum((content_act - generated_act)**2)/2 == get_content_loss(content_act, generated_act))

g = content_act[2,:] - generated_act[2, :]
g[[2, 4, 6, 9]]=0
assert(np.all(g==get_content_loss_gradient(content_act,generated_act)[2, :]))

# Testing the Style

In [111]:

np.random.seed(0)
content_act = np.random.uniform(size=(5, 5, 10)).astype(np.float32) - 0.5
np.random.seed(1)
generated_act = np.random.uniform(size=(5, 5, 10)).astype(np.float32) - 0.5


g_content = get_gram(content_act)
assert(np.abs(g_content[2,8] - np.sum(np.multiply(content_act[:,:,2].ravel(),content_act[:,:,8].ravel())))<1e-4)
assert(np.abs(g_content[4,5] - np.sum(np.multiply(content_act[:,:,4].ravel(),content_act[:,:,5].ravel())))<1e-4)


el_style = get_style_El(content_gram=get_gram(content_act), generated_gram=get_gram(generated_act), 
                        no_of_filters=content_act.shape[2], filter_size=content_act.shape[1]*content_act.shape[0])

assert(el_style - np.sum(np.square(get_gram(content_act) - get_gram(generated_act)))/(4*(25**2)*100)<1e-4)




In [ ]:
img1 = imread('Trumpfunny.jpg',  mode='RGB')
img1 = resize(img1, (224, 224))

prob = sess.run(vgg.probs, feed_dict={vgg.imgs: [img1]})[0]
preds = (np.argsort(prob)[::-1])[0:5]
for p in preds:
    print (class_names[p], prob[p])